In [11]:
from __future__ import print_function
from ipywidgets import Layout, Button, VBox, Label, Box, Checkbox, Image
import os
import shutil
from glob import glob
import matplotlib.pyplot as plt

In [16]:
_ROOT_DIR_REGEX = '/home/anpei/liury/data/facial-data/seg_face_real'


item_layout = Layout(overflow='scroll hidden',
                     border='3px solid orange',
                     display='flex', 
                     flex_flow='row',
                     align_items='stretch',
                     width='200px')


box_layout = Layout(overflow='scroll hidden',
                    border='3px solid black',
                    width='1500px',
                    flex_flow='row',
                    display='flex')


all_ins_dirs = sorted(glob(os.path.join(_ROOT_DIR_REGEX, './[0-9]*')))
cur_ins_idx = 0

def _build_vis():
    global cur_ins_idx
    global all_ins_dirs
    
    ins_dir = all_ins_dirs[cur_ins_idx % len(all_ins_dirs)]
    all_imgs = glob(os.path.join(ins_dir, 'seg_vis_*.png'))
    
    ins_id = os.path.basename(ins_dir)
    
    all_items = []
    
    for img_fp in all_imgs:
        with open(img_fp, 'rb') as f:
            img_raw = f.read()
            
        img_item = Image(value=img_raw)        
        img_item.layout.object_fit = 'scale-down'
        image_margin = '5 5 5 5'
    
        check_item = Checkbox(value=True, description=img_fp)
        
        def remove_item(change):
            check, img_fp = change['new'], change['owner'].description
            if not check and (os.path.exists(img_fp)):
                os.remove(img_fp)
                
        check_item.observe(remove_item, names='value')
        all_items.append(VBox([img_item, check_item]))
    
    label = Label('[ %s %s ] loaded %d items'%(('[C]' if cur_ins_idx > len(all_ins_dirs) else ''), ins_id, len(all_imgs)))
    carousel = Box(children=all_items, layout=box_layout)
    
    cur_ins_idx += 1
        
    return label, carousel


label, carousel = _build_vis()

next_btn = Button(description='NEXT')
output = VBox([next_btn,  label, carousel])

def next_folder(b):
    
    label, carousel = _build_vis()
    output.children = [b, label, carousel]

#         display(output)

next_btn.on_click(next_folder)

display(output)